## Description: Predictive Model, All Impressions
Each record is a unique user and campaign combination. Use only the first survey response per user & campaign.  And remove all impressions afterwards.  Convert the remaining impressions into separate columns with one cut per column, and the value signifying the frequency.

- Site/Channel/Device/Medium
- Baseline (Need to revisit this, basic comp)
- Month/Year/Weekday/Daytime
- Creative Size
- Recency
- Cum Sum of Weekly Impression Viewings


Notes:
- Product differences.
- Side by side comparisons of weighted vs. non-weighted data
- Frequency is very important.

Note: Need to correct the differences in sites, such as GDN vs. Google Display Network.

In [180]:
import pandas as pd, numpy as np, os 
from datetime import timedelta, datetime
from functools import reduce
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
%matplotlib inline

### Data Cleaning

In [181]:
df = pd.read_csv(r'C:\Users\william.raikes\Programming\Python\abacus\v1_model\data\raw\chromebook.csv')

In [182]:
def freq_counts(df, label):
    new_df = df.groupby(['user_id', 'campaign_id', label]).size().unstack().reset_index().fillna(0)  
    new_df.columns = [label+'_'+str(x) if x != 'user_id' and x != 'campaign_id' else x for x in new_df.columns]
    
    return new_df

In [183]:
def get_sample(df, start, end):
    tmp = df[
        (df.grp == 'CON') &
        (df.date >= start) &
        (df.date <= end)
    ]
    
    return tmp.shape[0]

In [184]:
def get_rate(df, start, end):
    tmp = df[
        (df.grp == 'CON') &
        (df.date >= start) &
        (df.date <= end)
    ].answer_desired_Aided_awareness
    
    return tmp.mean()

In [185]:
def update_baseline(group, df, sample, count):
    _days = 7 * count
    base = group.copy()
    
    for idx in group[group.baseline_n < sample].index:
        date = pd.to_datetime(group.loc[idx, 'baseline_month'])
        start = date - pd.DateOffset(days=_days)
        end = date + pd.DateOffset(months=1, days=_days)

        new_n = get_sample(df, start, end)
        new_rate = get_rate(df, start, end)

        base.loc[idx, 'baseline_n'] = new_n
        base.loc[idx, 'baseline'] = new_rate
        
    return base

In [186]:
def run_updates(base, sample):
    count = 1

    while any(base.baseline_n < sample):
        base = update_baseline(base, df, sample, count)
        count += 1
    
    return base

In [187]:
# Dates
df['date'] = pd.to_datetime(df.date)

df['survey_date'] = pd.to_datetime(df.Created_Aided_awareness_date)
df['recency'] = (df['survey_date'] - df['date']).dt.days

df['current_month'] = df.date.dt.to_period('M')
df['year'] = df['date'].dt.year
df['weekday'] = df['date'].dt.weekday
df['hour'] = df['date'].dt.hour
df['month'] = df['date'].dt.month
df['day_time'] = np.where((df.hour >= 6) & (df.hour < 12), 'AM',                                   #6am-11am
                    np.where((df.hour >= 12) & (df.hour < 18), 'Noon',                             #12pm-5pm
                        np.where((df.hour >= 18) & (df.hour < 23), 'PM',                           #6pm-10pm
                            np.where((df.hour >= 23) | (df.hour < 6), 'Late Night', np.nan))))     #11pm-5am

In [188]:
df['vid_size'] = df.dcm_ad.str.extract(r'(:[\d\d]{1,2}|[\d\d]{1,2}s)')[0].str.replace('s|:', '')
df['creative_size'] = np.where(df.creative_pixel_size == '0x0', 
                               df.vid_size, 
                               df.creative_pixel_size)

In [189]:
df['start_date'] = np.where(df.campaign_id==20334613, df[df.campaign_id==20334613].date.min(),
                       np.where(df.campaign_id==20570795, df[df.campaign_id==20570795].date.min(),
                           np.where(df.campaign_id==20920617, df[df.campaign_id==20920617].date.min(), np.nan)))

In [190]:
df['weeks_elapsed'] = ((df.date - df.start_date).dt.days // 7)

In [191]:
df['rolling_imps'] = 1
imps = df.sort_values('date').set_index('date').groupby(['user_id','campaign_id'])['rolling_imps'].rolling('7D').sum().reset_index()
df.drop(columns='rolling_imps', inplace=True)
df = pd.merge(df, imps, how='left', on=['user_id', 'campaign_id', 'date'])

In [192]:
base_sample = df.groupby('current_month').size().reset_index()
base_sample.columns = ['baseline_month', 'baseline_n']

base = df.groupby('current_month').mean().answer_desired_Aided_awareness.reset_index()
base.columns = ['baseline_month', 'baseline']

base = pd.merge(base, base_sample, on='baseline_month')
base.baseline_month = base.baseline_month.astype(str)

base.index = base.baseline_month
idx = pd.period_range(base.baseline_month.min(), base.baseline_month.max(), freq='M')
base = base.reindex(idx.astype(str), fill_value=0)
base.baseline_month = base.index
base.index = range(base.shape[0])

base = run_updates(base, 200)
base.columns = ['current_month', 'baseline', 'baseline_n']

In [193]:
df.current_month = df.current_month.astype(str)
df = pd.merge(df, base, how='left', on='current_month')

In [194]:
df.rolling_imps = np.where(df.rolling_imps >= 5, '5 plus', df.rolling_imps)
df.recency = np.where(df.recency >= 8, '8 plus', df.recency)

In [195]:
answer = df.groupby(['user_id', 'campaign_id'])['answer_desired_Aided_awareness'].max().reset_index()
campaign = df.groupby(['user_id', 'campaign_id']).olive_media_plan.first().reset_index()
grp = df.groupby(['user_id', 'campaign_id'])['grp'].first().reset_index()
frequency = df.groupby(['user_id', 'campaign_id'])['frequency'].first().reset_index()
baseline = df.sort_values(by=['user_id', 'campaign_id', 'date']).groupby(['user_id', 'campaign_id'])['baseline'].first().reset_index()

In [196]:
channel_name = freq_counts(df, 'channel_name')
device_name = freq_counts(df, 'device_name')
medium_name = freq_counts(df, 'medium_name')
weekday = freq_counts(df, 'weekday')
prst = freq_counts(df, 'prst')
rolling_imps = freq_counts(df, 'rolling_imps')
weeks = freq_counts(df, 'weeks_elapsed')
day_time = freq_counts(df, 'day_time')
creative_size = freq_counts(df, 'creative_size')
recency = freq_counts(df, 'recency')

In [197]:
dfs = [
    answer,
    campaign,
    grp,
    frequency,
    baseline,
    channel_name,
    device_name,
    medium_name,
    weekday,
    prst,
    rolling_imps,
    weeks,
    day_time,
    creative_size,
    recency
]

df_final = reduce(lambda left, right: pd.merge(left, right, how = 'inner', on=['user_id', 'campaign_id']), dfs)
df_final = pd.get_dummies(df_final, columns=['grp', 'olive_media_plan'])
df_final['total'] = df_final['channel_name_Display'] + df_final['channel_name_Video']

In [198]:
df_final.drop(columns=['user_id', 'campaign_id', 'grp_CON'], inplace=True)

In [199]:
cols = []
for col in df_final.columns:
    new_col = ''
    if ' ' in col:
        new_col = col.replace(' ', '_')
        cols.append(new_col)
    else:
        cols.append(col)

df_final.columns = cols

In [200]:
df_final.head()

,answer_desired_Aided_awareness,frequency,baseline,channel_name_Display,channel_name_Video,device_name_Desktop,device_name_Mobile,medium_name_Hybrid,medium_name_In_App,medium_name_Web,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,prst_ABC,prst_CBS_SPORTS,prst_CNET,prst_DBM,prst_FOX_NETWORKS,prst_FUSION_MEDIA_GROUP,prst_FUSION.NET,prst_GDN,prst_HUFFINGTON_POST,prst_HULU,prst_NBC,prst_NYTIMES,prst_OPEN_EXCHANGE_ADX,prst_OPEN_EXCHANGE_NONADX,prst_SAMBA,prst_VOX_MEDIA,prst_YOUTUBE,rolling_imps_1.0,rolling_imps_2.0,rolling_imps_3.0,rolling_imps_4.0,rolling_imps_5_plus,weeks_elapsed_0,weeks_elapsed_1,weeks_elapsed_2,weeks_elapsed_3,weeks_elapsed_4,weeks_elapsed_5,weeks_elapsed_6,weeks_elapsed_7,weeks_elapsed_8,weeks_elapsed_9,weeks_elapsed_10,weeks_elapsed_11,weeks_elapsed_12,day_time_AM,day_time_Late_Night,day_time_Noon,day_time_PM,creative_size_06,creative_size_15,creative_size_160x600,creative_size_30,creative_size_300x250,creative_size_300x600,creative_size_320x50,creative_size_336x280,creative_size_728x90,creative_size_970x250,creative_size_970x66,recency_0,recency_1,recency_2,recency_3,recency_4,recency_5,recency_6,recency_7,recency_8_plus,grp_EXP,olive_media_plan_Chromebooks_2018_NA_Q1_2018_North_America_-_Brand_&_DR_(US),olive_media_plan_Chromebooks_2018_NA_Q2_2018_North_America_-_Brand_(US),olive_media_plan_Chromebooks_NA_Q4_2017_United_States_-_Brand_&_DR,total
0,True,9,0.535844,4.0,0.0,4.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,4.0
1,False,4,0.604588,2.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0,0,1,0,2.0
2,False,17,0.700884,0.0,4.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0,1,0,0,4.0
3,True,16,0.574086,2.0,2.0,4.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0,1,4.0
4,True,4,0.574086,2.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1,0,0,1,2.0


#### This file is the base file; to be used to tree models.

In [201]:
df_final.to_csv(r'C:\Users\william.raikes\Programming\Python\abacus\v1_model\data\clean\explanatory\chromebook_all_imps_wo_ints.csv', index=False)

In [207]:
#20334613    
cols = [
    'channel_name_Display',
    'device_name_Desktop',
    'medium_name_Web',
    'prst_OPEN_EXCHANGE_ADX',
    'creative_size_300x250',
    'weekday_3',
    'rolling_imps_5_plus',
    'day_time_Late_Night',
    'creative_size_30',
    'recency_8_plus',
    'weeks_elapsed_6',
    'total',
    'olive_media_plan_Chromebooks_NA_Q4_2017_United_States_-_Brand_&_DR',
    'olive_media_plan_Chromebooks_2018_NA_Q1_2018_North_America_-_Brand_&_DR_(US)',
    'prst_YOUTUBE'
]

df_clean = df_final.drop(columns=cols)

#### This file removes the collinear features.

In [208]:
df_clean.to_csv(r'C:\Users\william.raikes\Programming\Python\abacus\v1_model\data\clean\explanatory\chromebook_all_imps_wo_ints.csv', index=False)

In [209]:
#df_clean = df_final.copy()
cols = [x for x in df_clean.columns if any([y in x for y in ['baseline', 'prst', 'channel', 'medium', 'device', 
                                                             'rolling', 'year', 'month', 'day_time',
                                                             'creative', 'recency', 'total', 'frequency', 'elapsed']])]

for col in cols:
    df_clean[col+'_int'] = df_clean[col] * df_clean['grp_EXP']

#### This file includes the interaction terms for all features; used for full model builds.

In [210]:
df_clean.to_csv(r'C:\Users\william.raikes\Programming\Python\abacus\v1_model\data\clean\explanatory\chromebook_all_imps_w_ints.csv', index=False)

In [211]:
cols = [x for x in df_clean.columns if '_int' in x or 'desired' in x or 'grp_EXP' in x]

df_int_only = df_clean[
    cols
]

#### This file includes only the interaction terms; used for partial stepwise model builds.

In [212]:
df_int_only.to_csv(r'C:\Users\william.raikes\Programming\Python\abacus\v1_model\data\clean\explanatory\chromebook_all_imps_ints_only.csv', index=False)

In [18]:
df.columns

Index(['grp', 'dcm_ad', 'olive_property', 'campaign_id', 'user_id', 'prst',
       'creative_pixel_size', 'date', 'answer_desired_Aided_awareness',
       'answer_desired_Consideration', 'answer_desired_Purchase_intent',
       'frequency', 'channel_name', 'device_name', 'medium_name',
       'olive_media_plan', 'Created_Aided_awareness_date', 'survey_date',
       'recency', 'current_month', 'year', 'weekday', 'hour', 'month',
       'day_time', 'vid_size', 'creative_size', 'start_date', 'weeks_elapsed',
       'rolling_imps', 'baseline', 'baseline_n'],
      dtype='object')

In [37]:
df.groupby(['user_id', 'campaign_id']).campaign_id.first()

user_id                      campaign_id
CAESEA-0zUhKmKudopgs2BG0N8o  20334613       20334613
CAESEA-Ax8Cc2zC_FoLsDq6sLN8  20920617       20920617
CAESEA-NIJV_UmFkNa_RDg1YI-Q  20570795       20570795
CAESEA-ncOmq5EOnrMaPvx5UFKw  20334613       20334613
CAESEA-pi6DfezXGehc3QKxRrV4  20334613       20334613
CAESEA-tJkY-4QAuTpqZJZpgi1Y  20334613       20334613
CAESEA01ulVilL5FBlD9d1L1egY  20334613       20334613
CAESEA06Qgl486rSxMMXDDbyqvo  20334613       20334613
CAESEA075WX-Zar5QV3wMC3-ZDc  20334613       20334613
CAESEA07b2dYOD7UdzrA9D-6w-Y  20570795       20570795
CAESEA09KdCHQxYfuKyvZdh9MNg  20570795       20570795
CAESEA0CxO7Xb1fBWNjYmjOMYwE  20334613       20334613
CAESEA0F6z4ICrmbmNOzImNpdmY  20334613       20334613
CAESEA0jTTge86h0o-vS8dnCh8k  20334613       20334613
CAESEA0scZsis0RkaAPWK1_75-Y  20334613       20334613
CAESEA114QjoC584aIs4fWovzow  20334613       20334613
CAESEA15rhYSfY5NAmv8c7_k9rs  20920617       20920617
CAESEA1FvtxPuhkCE_bL7SQKZY4  20334613       20334613
CAESE

In [20]:
for col in df.columns:
    if 'user_id' in col or 'dcm_ad' in col or 'prst' in col or 'date' in col or 'frequency' in col:
        pass
    else:
        print(df[col].value_counts(), end='\n\n')

CON    21769
EXP    21633
Name: grp, dtype: int64

Open Exchange AdX       13370
YouTube                 11780
Open Exchange NonAdX     6720
GDN                      5588
DBM                      1720
fusion.net               1323
Fusion Media Group        987
Vox Media                 464
CNet                      404
CBS Sports                386
Huffington Post           303
NYTimes                   217
Samba                      45
Hulu                       36
NBC                        26
ABC                        23
Fox Networks               10
Name: olive_property, dtype: int64

20334613    29021
20570795     7873
20920617     6508
Name: campaign_id, dtype: int64

0x0        15241
300x250    12944
970x250     4148
728x90      3607
300x600     2642
320x50      2622
160x600     1349
336x280      715
970x66       134
Name: creative_pixel_size, dtype: int64

True     25103
False    18299
Name: answer_desired_Aided_awareness, dtype: int64

False    33777
True      9625
Name: answ

In [356]:
df[df.grp == 'EXP'].groupby('weeks_elapsed').answer_desired_Aided_awareness.agg(['mean', 'size'])

,mean,size
weeks_elapsed,,
0,0.475822,517
1,0.551364,1246
2,0.597954,1271
3,0.637568,1661
4,0.574599,811
5,0.625922,2847
6,0.604907,4280
7,0.524983,2922
8,0.533679,1930


In [243]:
df[df.grp == 'EXP'].groupby('rolling_imps').answer_desired_Aided_awareness.agg(['mean', 'size'])

,mean,size
rolling_imps,,
1.0,0.591741,6175
2.0,0.583510,3784
3.0,0.584390,2524
4.0,0.572387,1789
5.0,0.575507,1331
6.0,0.573880,1049
7.0,0.580882,816
8 plus,0.576711,4165


In [246]:
df.groupby(['year', 'month']).baseline.agg(['mean', 'size'])

mean   size
year month                 
2017 10     0.490270   3443
     11     0.535844  16488
     12     0.574086   9084
2018 1      0.526000    500
     2      0.700884   5884
     3      0.687625   1495
     5      0.629213    623
     6      0.604588   5885